In [2]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

from tqdm import tqdm

In [ ]:
#Creo la BBDD:

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE proyecto_cine")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

----
**API**

In [3]:
#quitar duplicados de pelis

df = pd.read_csv('pelis_ordenadas.csv')

# Eliminar duplicados
new_df_no_duplicate = df.drop_duplicates(subset=['ID'])

# Guardar el DataFrame modificado en un nuevo archivo CSV
new_df_no_duplicate.to_csv('pelis_ordenadas_sin_duplicados.csv', index=False)

In [4]:

df = pd.read_csv('pelis_ordenadas_sin_duplicados.csv')

In [5]:
df

,Unnamed: 0,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
0,0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874
1,16871,Comedy,Movie,Bad boy dak gung,2000,12.0,tt0284878
2,16872,Comedy,Movie,Dead or Alive 2: Tôbôsha,2000,12.0,tt0285005
3,16873,Comedy,Movie,Something Between Us,2000,12.0,tt0286137
4,16874,Comedy,Movie,Fughe da fermo,2000,9.0,tt0285122
...,...,...,...,...,...,...,...
524315,6306,Drama,Short,Simon x Lucy Single Scene Short (Sx3) VVV (Par...,2023,5.0,tt29793270
524316,6305,Drama,Short,Melanie x Diya 1 Single Scene Shorts (Sx3) VVV...,2023,3.0,tt29792944
524317,6304,Drama,Short,Desgosto de morrer no Inverno,2023,NaN,tt29792241
524318,6320,Drama,Short,Divebomb,2023,4.0,tt29808719


In [6]:
# Creo la tabla para las películas

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS peliculas (
    Genero VARCHAR(255),
    Tipo VARCHAR(255),
    Titulo VARCHAR(255),
    Anio_de_estreno INT,
    Mes_de_estreno FLOAT,
    ID VARCHAR(255) PRIMARY KEY
)
"""

# Ejecuta la instrucción SQL para crear la tabla
try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)

# Cierra el cursor y la conexión a la base de datos
mycursor.close()
cnx.close()

Tabla creada correctamente.


In [7]:
# Insertar información de las películas

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()

# Lee el archivo CSV
df = pd.read_csv("pelis_ordenadas_sin_duplicados.csv")
df.drop(df.columns[0], axis=1, inplace=True) #elimina la primera columna

# Define el rango de filas sobre el cual iterar
inicio = 0  
fin = 524320

# Itera sobre el rango de filas especificado del DataFrame
for index in tqdm(range(inicio, fin), desc="Insertando registros"):
    row = df.iloc[index]  # Obtén la fila en el índice actual
    sql = "INSERT INTO peliculas (Genero, Tipo, Titulo, Anio_de_estreno, Mes_de_estreno, ID) VALUES (%s, %s, %s, %s, %s, %s)"
    # Convierte el año de estreno y el mes a tipo int
    año_de_estreno = int(row['Año de estreno']) if not pd.isnull(row['Año de estreno']) else None
    mes_de_estreno = float(row['Mes de estreno']) if not pd.isnull(row['Mes de estreno']) else None
    val = (row['Genero'], row['Tipo'], row['Titulo'], año_de_estreno, mes_de_estreno, row['ID'])
    try:
        mycursor.execute(sql, val)
        cnx.commit()  # Confirma la transacción
    except mysql.connector.Error as err:
        cnx.rollback()  # Revierte la transacción si hay un error
        print("Error al insertar registro:", err)

# Cierra el cursor y la conexión a la base de datos
mycursor.close()
cnx.close()

Insertando registros:  13%|█▎        | 66578/524321 [01:44<11:20, 672.79it/s]

Error al insertar registro: 1406 (22001): Data too long for column 'Titulo' at row 1


Insertando registros: 100%|█████████▉| 524320/524321 [13:19<00:00, 655.49it/s]


IndexError: single positional indexer is out-of-bounds

----
**IMDB Y ROTTEN**

In [8]:
#quitar duplicados de pelis

df = pd.read_csv('INFO_IMBD_API.csv')

# Eliminar duplicados
new_df_no_duplicate = df.drop_duplicates(subset=['ID'])

# Guardar el DataFrame modificado en un nuevo archivo CSV
new_df_no_duplicate.to_csv('IMBD_sin_duplicados.csv', index=False)

In [9]:
df = pd.read_csv('IMBD_sin_duplicados.csv')

In [10]:
df

,Unnamed: 0,Titulo,ID,Puntuacion_IMDB,Dirección,Guionistas,Argumento,Duración,puntuacion
0,0,Chateaubriand - Cabeça de Paraíba,tt0115874,--,['Marcos Manhães Marins'],['Marcos Manhães Marins'],NaN,15min,No existe puntuacion
1,1,Bad boy dak gung,tt0284878,"4,8",['Wai-Man Yip'],['Manfred Wong'],Badboy is a HK detective agency. They speciali...,1h 45min,No existe puntuacion
2,7,Dead or Alive 2: Tôbôsha,tt0285005,NaN,NaN,NaN,NaN,NaN,NaN
3,8,Something Between Us,tt0286137,"8,6",['George Thomas Jr.'],['George Thomas Jr.'],NaN,1h 19min,No existe puntuacion
4,9,Fughe da fermo,tt0285122,"6,1",['Edoardo Nesi'],DESCONOCIDO,NaN,1h 32min,No existe puntuacion
...,...,...,...,...,...,...,...,...,...
524315,964178,Simon x Lucy Single Scene Short (Sx3) VVV (Par...,tt29793270,NaN,NaN,NaN,NaN,NaN,NaN
524316,964179,Melanie x Diya 1 Single Scene Shorts (Sx3) VVV...,tt29792944,NaN,NaN,NaN,NaN,NaN,NaN
524317,964180,Desgosto de morrer no Inverno,tt29792241,NaN,NaN,NaN,NaN,NaN,NaN
524318,964181,Divebomb,tt29808719,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Creo la tabla para IMDB

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS peliculas_info (
    Titulo VARCHAR(255),
    ID VARCHAR(255),
    Puntuacion_IMDB VARCHAR(255),
    Direccion VARCHAR(255),
    Guionistas VARCHAR(255),
    Argumento TEXT,
    Duracion VARCHAR(255),
    Puntuacion_Rotten VARCHAR(255),
    FOREIGN KEY (ID) REFERENCES peliculas (ID) ON DELETE CASCADE ON UPDATE CASCADE
)
"""
# Puntuacion_IMDB, Direccion, Guionistas, Argumento, Duracion, Nombre_de_la_pelicula

try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)


mycursor.close()
cnx.close()

Tabla creada correctamente.


In [16]:
#Insertar información IMDB

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


df = pd.read_csv("IMBD_sin_duplicados.csv")
df.drop(df.columns[0], axis=1, inplace=True)


inicio = 0  
fin = 524320   


for index in tqdm(range(inicio, fin), desc="Insertando registros"):
    row = df.iloc[index]  
    sql = "INSERT INTO peliculas_info (Titulo, ID, Puntuacion_IMDB, Direccion, Guionistas, Argumento, Duracion, Puntuacion_Rotten) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
   
    try:
        mycursor.execute(sql, (str(row['Titulo']), str(row['ID']), str(row['Puntuacion_IMDB']), str(row['Dirección']), str(row['Guionistas']), str(row['Argumento']), str(row['Duración']), str(row['puntuacion']),))
        cnx.commit()  
    except mysql.connector.Error as err:
        cnx.rollback()  
        print("Error al insertar registro:", err)


mycursor.close()
cnx.close()

Insertando registros:  13%|█▎        | 66536/524320 [02:18<17:13, 442.83it/s]

Error al insertar registro: 1406 (22001): Data too long for column 'Titulo' at row 1


Insertando registros: 100%|██████████| 524320/524320 [15:19<00:00, 570.39it/s]


-----
**ACTORES**

In [18]:
#quitar duplicados de actores

df = pd.read_csv('union_actores_unicos.csv')

# Eliminar duplicados
new_df_no_duplicate = df.drop_duplicates()

# Guardar el DataFrame modificado en un nuevo archivo CSV
new_df_no_duplicate.to_csv('actores_sin_duplicados.csv', index=False)

In [17]:
# Creo la tabla actores

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS info_actores (
    Nombre VARCHAR(255),
    Nacimiento VARCHAR(255),
    Conocido VARCHAR(255),
    Que_hace TEXT,
    Premios VARCHAR(255),
    Oscar VARCHAR(255)
)
"""
# Nombre,Nacimiento,Conocido,Que hace,Premios,Oscar

try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)


mycursor.close()
cnx.close()

Tabla creada correctamente.


In [20]:
#Insertar información IMDB

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


df = pd.read_csv("actores_sin_duplicados.csv")
df.drop(df.columns[0], axis=1, inplace=True)

inicio = 0
fin = 36981


for index in tqdm(range(inicio, fin)):
   
    row = df.iloc[index]
    
    sql = "INSERT INTO info_actores (Nombre, Nacimiento, Conocido, Que_hace, Premios, Oscar) VALUES (%s, %s, %s, %s, %s, %s)"
    
    try:
        mycursor.execute(sql, (str(row['Nombre']), str(row['Nacimiento']), str(row['Conocido']), str(row['Que hace']), str(row['Premios']), str(row['Oscar'])))
        cnx.commit() 
    except mysql.connector.Error as err:
        cnx.rollback()  
        print("Error al insertar registro:", err)


mycursor.close()
cnx.close()

 95%|█████████▌| 35303/36981 [01:18<00:02, 710.90it/s]

Error al insertar registro: 1406 (22001): Data too long for column 'Conocido' at row 1


100%|██████████| 36981/36981 [01:21<00:00, 454.77it/s]
